In [1]:
from transformers import PreTrainedTokenizerFast, DataCollatorForLanguageModeling, PhiConfig, PhiForCausalLM, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset
import pandas as pd
import time
import torch

# 1. 数据来源，保存路径，最大长度定义

In [2]:
tokenizer_dir = './model_save/tokenizer/'
model_save_dir = './model_save/pre/'
logs_dir = './logs/'
train_file = './data/instruction_data.parquet'
max_seq_len = 320

# 2. 加载训练好的tokenizer
如果你使用的`add_tokens`方法添加了自己的token，必须要用`len(tokenizer)`获取长度，`tokenizer.vocab_size`统计不包含你添加的字符。

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_dir)
print(f"vicab size: {len(tokenizer)}")

vicab size: 35840


In [4]:
# tokenizer.bos_token = '[BOS]'
# tokenizer.eos_token = '[PAD]'
# tokenizer.unk_token = '[UNK]'
# tokenizer.mask_token = '[MASK]'
# tokenizer.pad_token = '[PAD]'
# tokenizer.cls_token = '[CLS]'
# tokenizer.save_pretrained(tokenizer_dir)

# 3. 加载数据集

In [4]:
dataset = load_dataset(path='parquet', data_files=train_file, split='train', cache_dir='.cache')

In [ ]:
dataset

In [6]:
samples = dataset['text'][0:5]
print(samples)

['判断给定的文章是否符合语法规则。如果不符合，请提供修改建议。\n下面是一篇文章的开头: "为了探讨这个主题，本文将提供一系列数据和实例，以证明这一观点。"\n这个开头符合语法规则。', '提供一个包含两个城市名称的列表，然后为这两个城市之间的距离提供一个近似值。\n1. 北京\n2. 上海\n北京和上海之间的距离约为 1,295 公里。', '解释人类尺度在绘画中的作用。\n人类尺度在绘画中的作用是用来呈现逼真的透视和比例。通过考虑物体与人类身体的关系，艺术家可以创建出真实感和深度感，使观众更好地理解画作中的场景和物品。人类尺度可以让画作更加可信，并且帮助观众更容易地投入到画作的世界中。', '给定一篇文章，纠正里面的语法错误。\n我去年很喜欢在公园里跑步，但因为最近天气太冷所以我不去了。\n我去年很喜欢在公园里跑步，但由于最近天气太冷，所以我不去了。', '根据以下输入生成相应的段落：\n[组织名]于今天宣布将收购[公司名]。该交易总价为[$总价]。[公司名]将继续保持独立，并在以前的管理团队领导下运营。今天，[组织名]宣布他们将收购[公司名]，该交易总价为[$总价]。尽管发生了这样的变化，[公司名]将继续保持独立，继续保留以前的管理团队领导，并继续运营。']


## token to id缓存到文件，使用的时候不用再次tokenize

In [7]:
def token_to_id(samples: dict[str, list]) -> dict:
    batch_txt = []
    for txt in samples['text']:
        batch_txt.append(
            f"[BOS]{txt}[EOS]"
        )
    outputs = tokenizer(
        batch_txt,
        truncation=False,
        padding=False,
    )

    return {
        "input_ids": outputs["input_ids"], 
        }

print(token_to_id({'text':['判断给定的文章是否符合语法规则。如果不符合，请提供修改建议。\n','下面是一篇文章的开头: "为了探讨这个主题，本文将提供一系列数据和实例，以证明这一观点。']}))


{'input_ids': [[3, 1948, 1829, 1128, 8739, 20114, 225, 678, 10254, 221, 3985, 32137, 225, 177, 1], [3, 1933, 547, 10085, 24782, 32, 1596, 1791, 3259, 17514, 221, 5187, 424, 541, 4991, 7940, 9144, 221, 255, 3135, 2479, 2127, 225, 1]]}


In [ ]:

tokenized_datasets = dataset.map(
    token_to_id, batched=True, remove_columns=dataset.column_names
)
tokenized_datasets

# 4. 定义data_collator
`mlm=False`表示要训练CLM模型，`mlm=True`表示要训练MLM模型

In [9]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [10]:
few_data = [tokenized_datasets[i] for i in range(5)]

In [12]:
# print(few_data)

##  验证一下数据看padding、输入输出是否符合要求

In [11]:
out = data_collator(few_data)
print(out.keys())
for key in out:
    # print(out[key])
    print(f"{key} shape: {out[key].shape}")

# input_ids 和 labels 相同
sum(out['input_ids'][0] == out['labels'][0]) == sum(out['attention_mask'][0])

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels'])
input_ids shape: torch.Size([5, 89])
attention_mask shape: torch.Size([5, 89])
labels shape: torch.Size([5, 89])


tensor(True)

In [14]:
# print(out['labels'][0])
# print(out['attention_mask'][0])

# 5. 定义模型
从`config`定义，不是`from_pretrained`。 
为了方便cuda计算，词表的大小注意一下，如果不是64的整数倍，可以手动向上取整为64的整数倍，也可以是其他 $2^x$ 数值的整数倍，如32、128、256都行。

In [12]:
vocab_size = len(tokenizer)
if vocab_size % 64 != 0:
    vocab_size = (vocab_size // 64 + 1) * 64
print(f"final vocab sieze: {vocab_size}")

final vocab sieze: 35840


In [13]:
phi_config = PhiConfig(
    vocab_size=vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=16,
    max_position_embeddings=512,
    intermediate_size=4096,
)

model = PhiForCausalLM(phi_config)

model_size = sum(t.numel() for t in model.parameters())
print(f"Phi-2 size: {model_size / 1000**2:.1f}M parameters")

Phi-2 size: 193.7M parameters


# 6. cuda cache回调函数

In [14]:
class EmptyCudaCacheCallback(TrainerCallback):
    log_cnt = 0
    def on_log(self, args, state, control, logs=None, **kwargs):
        self.log_cnt += 1
        if self.log_cnt % 5 == 0:
            torch.cuda.empty_cache()
            
empty_cuda_cahce = EmptyCudaCacheCallback()

# 6. 定义训练参数

In [ ]:
args = TrainingArguments(
    output_dir=model_save_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    num_train_epochs=4,
    weight_decay=0.1,
    warmup_steps=1000,
    learning_rate=5e-4,
    save_steps=2000,
    save_total_limit=3,
    report_to='tensorboard',
    optim="adafactor",
    bf16=True,
    logging_steps=10,
    log_level='info',
    logging_first_step=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
    callbacks=[empty_cuda_cahce],
)

# 7. 开始训练
`resume_from_checkpoint=True`参数可以从上次保存的检查点继续训练

In [ ]:
trainer.train(
    # resume_from_checkpoint=True
)

# 8. 最后保存训练的loss日志和模型

In [18]:

loss_log = pd.DataFrame(trainer.state.log_history)
loss_log.to_csv(f"./logs/pre_train_log_{time.strftime('%Y%m%d-%H%M')}.csv")


trainer.save_model(model_save_dir)

Saving model checkpoint to ./model_save/pre/
Configuration saved in ./model_save/pre/config.json
Configuration saved in ./model_save/pre/generation_config.json
Model weights saved in ./model_save/pre/pytorch_model.bin
tokenizer config file saved in ./model_save/pre/tokenizer_config.json
Special tokens file saved in ./model_save/pre/special_tokens_map.json
